# Ingesta de datos - Fase 0
## DSRP - Especialización de Machine Learning -  Curso MLOps e Ingeniería de Software para ML


In [12]:
import os
import urllib
import orjson

import polars as pl

import aiohttp
import asyncio
from pathlib import Path

from typing import List, Dict
from loguru import logger


from dotenv import load_dotenv

In [17]:
if not os.path.isdir("data"):
    os.mkdir("./data", "data/")

## Descarga Datasets Abiertos

In [18]:
urllib.request.urlretrieve(
    "https://datasets.imdbws.com/title.basics.tsv.gz", 
    "data/title.basics.tsv.gz" 
)

('data/title.basics.tsv.gz', <http.client.HTTPMessage at 0x111e03d50>)

In [4]:
basics = pl.read_csv(
    "data/title.basics.tsv.gz",
    separator="\t",
    null_values=["\\N"],
    quote_char=None
)

In [27]:
urllib.request.urlretrieve(
    "https://datasets.imdbws.com/title.ratings.tsv.gz", 
    "data/title.ratings.tsv.gz" 
)

('data/title.ratings.tsv.gz', <http.client.HTTPMessage at 0x12202e010>)

In [5]:
ratings = pl.read_csv(
    "data/title.ratings.tsv.gz",
    separator="\t",
    null_values=["\\N"],
    quote_char=None
)

## DataSet Limpio

In [6]:
movies = (
    basics
    .filter(pl.col("titleType") == "movie")
    .select([
        pl.col("tconst").alias("imdb_id"),
        pl.col("primaryTitle").alias("title"),
        pl.col("startYear").cast(pl.Int32).alias("year"),
        pl.col("genres")
    ])
    .join(
        ratings.select(["tconst", "averageRating", "numVotes"]),
        left_on="imdb_id",
        right_on="tconst",
    )
    .rename({
        "averageRating": "imdb_rating",
        "numVotes": "imdb_votes",
    })
    .filter(
        (pl.col("imdb_votes") >= 1000)
    )
)

In [13]:
movies

imdb_id,title,year,genres,imdb_rating,imdb_votes
str,str,i32,str,f64,i64
"""tt0000574""","""The Story of the Kelly Gang""",1906,"""Action,Adventure,Biography""",6.0,1039
"""tt0002130""","""Dante's Inferno""",1911,"""Adventure,Drama,Fantasy""",7.0,3989
"""tt0002423""","""Passion""",1919,"""Biography,Drama,Romance""",6.7,1105
"""tt0002844""","""Fantômas: In the Shadow of the…",1913,"""Crime,Drama""",6.9,2686
"""tt0003014""","""Ingeborg Holm""",1913,"""Drama""",7.0,1567
…,…,…,…,…,…
"""tt9907782""","""The Cursed""",2021,"""Fantasy,Horror,Mystery""",6.2,22054
"""tt9908390""","""Le lion""",2020,"""Comedy""",5.5,1600
"""tt9911196""","""The Marriage Escape""",2020,"""Comedy,Drama""",7.4,3520


In [7]:
with open("data/movie_ids.txt", "w") as f:
    for item in movies["imdb_id"].to_list():
        f.write(f"{item}\n")

## 2. Datos OMDB API

In [8]:
load_dotenv()

# -----------------------------
# Config
# -----------------------------
OMDB_API_URL = "https://www.omdbapi.com/"
CONCURRENCY = 20
MAX_RETRIES = 5
CHECKPOINT_FILE = "data/processed_ids.txt"

# -----------------------------
# Loguru setup (minimal, colored)
# -----------------------------
logger.remove()
logger.add(
    lambda msg: print(msg, end=""),
    colorize=True,
    format="<green>{time:HH:mm:ss}</green> | "
           "<level>{level}</level> | "
           "{message}\n",
)


# -----------------------------
# Checkpoint utilities
# -----------------------------
def load_processed_ids() -> set[str]:
    """Load already-processed IMDb IDs from checkpoint file."""
    if Path(CHECKPOINT_FILE).exists():
        logger.info(f"Loading checkpoint from {CHECKPOINT_FILE}")
        with open(CHECKPOINT_FILE, "r") as f:
            return set(line.strip() for line in f if line.strip())
    return set()


def append_processed_id(imdb_id: str) -> None:
    """Append a single IMDb ID to checkpoint file."""
    with open(CHECKPOINT_FILE, "a") as f:
        f.write(imdb_id + "\n")


# -----------------------------
# Single OMDb request
# -----------------------------
async def fetch_one(session: aiohttp.ClientSession, imdb_id: str) -> Dict | None:
    """
    Fetch a single OMDb record:
    - by IMDb ID (param i)
    - with plot=full
    Returns dict: {"imdb_id": ..., "raw": <omdb_json_dict>} or None on final failure.
    """
    params = {
        "i": imdb_id,
        "apikey": os.getenv("OMDB_API_KEY"),
        "plot": "full",
    }

    retries = 0
    while retries < MAX_RETRIES:
        try:
            async with session.get(OMDB_API_URL, params=params) as resp:
                status = resp.status

                if status == 429:
                    wait = 5 * (retries + 1)
                    logger.warning(f"[{imdb_id}] 429 Too Many Requests — sleeping {wait}s")
                    await asyncio.sleep(wait)
                    retries += 1
                    continue

                if status >= 500:
                    wait = 3 * (retries + 1)
                    logger.warning(f"[{imdb_id}] Server error {status} — retrying in {wait}s")
                    await asyncio.sleep(wait)
                    retries += 1
                    continue

                data = await resp.json()

                # OMDb logical error (invalid ID, etc.)
                if data.get("Response") == "False":
                    err = data.get("Error", "Unknown error")
                    logger.error(f"[{imdb_id}] OMDb error: {err}")
                    return {"imdb_id": imdb_id, "raw": data}

                # Success (no log to avoid spam)
                return {"imdb_id": imdb_id, "raw": data}

        except asyncio.TimeoutError:
            retries += 1
            wait = 2 * retries
            logger.warning(f"[{imdb_id}] Timeout — retrying in {wait}s")
            await asyncio.sleep(wait)

        except Exception as e:
            retries += 1
            wait = 2 * retries
            logger.error(f"[{imdb_id}] Unexpected error: {e} — retrying in {wait}s")
            await asyncio.sleep(wait)

    logger.critical(f"[{imdb_id}] FAILED permanently after {MAX_RETRIES} retries")
    return None


# -----------------------------
# Worker: pulls IDs from queue, writes JSONL incrementally
# -----------------------------
async def worker(
    worker_id: int,
    session: aiohttp.ClientSession,
    queue: asyncio.Queue,
    jsonl_path: Path,
    write_lock: asyncio.Lock,
):
    while True:
        imdb_id = await queue.get()
        if imdb_id is None:
            queue.task_done()
            return

        result = await fetch_one(session, imdb_id)

        if result is not None:
            # Persist JSON immediately (JSONL) to avoid losing data on crash
            line = orjson.dumps(result).decode("utf-8") + "\n"

            # Ensure writes are atomic-ish using a lock
            async with write_lock:
                with open(jsonl_path, "a", encoding="utf-8") as f:
                    f.write(line)

            append_processed_id(imdb_id)

        queue.task_done()


# -----------------------------
# Main async pipeline
# -----------------------------
async def main(
    all_ids: List[str],
    output_file: str = "omdb_raw.jsonl",
):
    """
    Main entrypoint for the downloader.

    - all_ids: list of IMDb IDs (ttXXXXXX)
    - output_file: path for the JSONL file

    This function:
    - skips IDs already in the checkpoint
    - fetches remaining IDs with concurrency
    - persists each result as one JSON line to output_file
    - is safe to rerun after crashes
    """
    jsonl_path = Path(output_file)

    processed = load_processed_ids()
    remaining = [i for i in all_ids if i not in processed]

    logger.info(f"Total IDs: {len(all_ids)}")
    logger.info(f"Already processed: {len(processed)}")
    logger.info(f"Remaining to fetch: {len(remaining)}")
    logger.info(f"Output JSONL: {jsonl_path}")

    if not remaining:
        logger.success("Nothing to do. All IDs are already processed.")
        return

    queue: asyncio.Queue = asyncio.Queue()
    write_lock = asyncio.Lock()

    # Enqueue all remaining IDs
    for imdb_id in remaining:
        await queue.put(imdb_id)

    timeout = aiohttp.ClientTimeout(total=40)

    async with aiohttp.ClientSession(timeout=timeout) as session:
        # Start workers
        workers = [
            asyncio.create_task(worker(i, session, queue, jsonl_path, write_lock))
            for i in range(CONCURRENCY)
        ]

        # Add termination signals for each worker
        for _ in workers:
            await queue.put(None)

        # Wait until all tasks are done
        await queue.join()

        # Cancel any stray workers
        for w in workers:
            w.cancel()

    logger.success("Download complete.")
    logger.info(f"Results appended to {jsonl_path}")


In [9]:
with open("data/movie_ids.txt") as f:
    all_ids = [line.strip() for line in f if line.strip()]

# Run the async downloader
await main(all_ids, output_file="data/omdb_raw.jsonl")

18:56:37 | INFO | Loading checkpoint from data/processed_ids.txt

18:56:37 | INFO | Total IDs: 47203

18:56:37 | INFO | Already processed: 100

18:56:37 | INFO | Remaining to fetch: 47103

18:56:37 | INFO | Output JSONL: data/omdb_raw.jsonl

19:01:04 | ERROR | [tt38018909] OMDb error: Incorrect IMDb ID.

19:01:04 | ERROR | [tt38077500] OMDb error: Incorrect IMDb ID.

19:01:04 | ERROR | [tt38063392] OMDb error: Incorrect IMDb ID.

19:01:04 | ERROR | [tt38095263] OMDb error: Incorrect IMDb ID.

19:01:04 | ERROR | [tt38121182] OMDb error: Incorrect IMDb ID.

19:01:04 | ERROR | [tt38162716] OMDb error: Incorrect IMDb ID.

19:01:04 | ERROR | [tt38202962] OMDb error: Incorrect IMDb ID.

19:01:05 | ERROR | [tt38235319] OMDb error: Incorrect IMDb ID.

19:01:05 | ERROR | [tt38367304] OMDb error: Incorrect IMDb ID.

19:01:05 | ERROR | [tt38394613] OMDb error: Incorrect IMDb ID.

19:01:05 | ERROR | [tt38428491] OMDb error: Incorrect IMDb ID.

19:01:05 | ERROR | [tt38428591] OMDb error: Incorrect 

In [11]:
df = pl.read_ndjson("data/omdb_raw.jsonl")
df

shape: (47_203, 2)
┌───────────┬─────────────────────────────────┐
│ imdb_id   ┆ raw                             │
│ ---       ┆ ---                             │
│ str       ┆ struct[25]                      │
╞═══════════╪═════════════════════════════════╡
│ tt0002423 ┆ {"Passion","1919","Not Rated",… │
│ tt0004181 ┆ {"Judith of Bethulia","1914","… │
│ tt0004465 ┆ {"The Perils of Pauline","1914… │
│ tt0003643 ┆ {"The Avenging Conscience: or … │
│ tt0002844 ┆ {"Fantômas: In the Shadow of t… │
│ …         ┆ …                               │
│ tt9907782 ┆ {"The Cursed","2021","R","18 F… │
│ tt9908390 ┆ {"Le lion","2020","N/A","29 Ja… │
│ tt9911196 ┆ {"The Marriage Escape","2020",… │
│ tt9916362 ┆ {"Coven","2020","TV-MA","02 Oc… │
│ tt9916270 ┆ {"Il talento del calabrone","2… │
└───────────┴─────────────────────────────────┘